# Search stocks

In [ ]:
import sys
!{sys.executable} -m pip install pandas_datareader yfinance matplotlib

In [ ]:
import pandas as pd
from pandas import DataFrame
import json
from pandas_datareader import data as pdr
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt



In [ ]:
yf.pdr_override()
#  https://datahub.io/core/nasdaq-listings
df = pd.read_csv('https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed_csv/data/7665719fb51081ba0bd834fde71ce822/nasdaq-listed_csv.csv')
# convert symbol column to list

tickers = df['Symbol'].values.tolist()
#t = [x.replace('.', '-') for x in tickers] 
#print(tickers)
tickers
#yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")

In [ ]:
ma_count = 21

print(dt.datetime(2021,1,1))
data = pdr.get_data_yahoo(tickers, start=dt.datetime(2021,1,1), end=dt.date.today())['Close']
data.tail()


[******************************138%********************************]  4083 of 2967 completed

In [ ]:
ma = data.rolling(window=ma_count).mean()
ma_pers = data.div(ma).mul(100).sub(100)
#ma_pers.iloc[[-1]]
ma_pers_r=ma_pers.iloc[::-1]
d = {}
df_prep = []
for col_name in ma_pers_r.columns:
    d[col_name]=0
    ma_last = None
    for index, row in ma_pers_r.iterrows():
        if row[col_name] > 0:
            d[col_name] += 1
            if ma_last == None:
                ma_last=row[col_name]
        else:
            if d[col_name] != 0:
               df_prep.append([col_name, d[col_name], ma_last, ma_count, "daily"]) 
            break
df = DataFrame(df_prep,columns=['name','count', 'ma_diff', 'ma_size', 'ma_type']).sort_values(by=['ma_diff', 'count'], ascending=False)
print(df['count'][df['count'] >= 10].size)
lead_comps_df = df[df['count'] >= 10]
lead_comps = lead_comps_df['name'].values.tolist()
lead_comps_df

In [ ]:
for quote in lead_comps:
    fig, ax = plt.subplots(figsize=(20, 10))
    plt.plot(data[quote], label=quote); 
    plt.plot(ma[quote], label = 'MA');
    plt.legend(loc=2);